# 🔎 VerificAI - Uma IA de suporte e análise de currículos!

## Instalações, importações e requisições necessárias para a execução correta do programa.

In [21]:
%pip -q install google-generativeai
%pip -q install google-adk
!pip install PyPDF2

import os
import sys
from google.colab import files, userdata
import PyPDF2
from google import genai
from IPython.display import display, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

Conexão agente - parâmetro

In [22]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return final_response.strip()


In [23]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Definições dos agentes e suas respectivas *características*

In [35]:
def agente_avaliador(curriculo_extraido: str):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista sênior em Recursos Humanos, com vasta experiência em avaliação de currículos.
        Sua tarefa é analisar o conteúdo abaixo de forma crítica e construtiva.

        Siga as diretrizes a seguir:

        1. Identifique e destaque os principais pontos fortes do candidato, incluindo habilidades técnicas, experiências relevantes e conquistas notáveis.
        2. Aponte de forma clara e respeitosa os aspectos que podem ser aprimorados, como organização, clareza, formatação, uso de palavras-chave ou linguagem profissional.
        3. Mantenha uma abordagem objetiva, profissional e direta. Não há necessidade de cumprimento.

        Evite generalizações. Foque em observações específicas e úteis que realmente possam ajudar o candidato a melhorar seu currículo.
        """,
        description="Agente que avalia currículos criticamente."
    )
    entrada = f"Currículo:\n{curriculo_extraido}"
    return call_agent(avaliador, entrada)

In [36]:
def agente_otimizador(curriculo_extraido: str):
    otimizador = Agent(
        name="agente_otimizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em revisão e otimização de currículos, com ampla experiência em Recursos Humanos e comunicação profissional.

        Revise o conteúdo abaixo com o objetivo de torná-lo mais claro, direto e impactante. Faça os seguintes ajustes:
        - Corrija erros gramaticais, ortográficos e de pontuação.
        - Reescreva trechos confusos ou ambíguos, aprimorando a fluidez do texto.
        - Adote uma linguagem profissional e assertiva, adequada ao contexto de um currículo competitivo.
        - Mantenha as informações relevantes, destacando conquistas, competências e experiências de forma estratégica.

        O texto resultante deve transmitir profissionalismo, credibilidade e destacar o potencial do candidato.
        """,
        description="Agente otimizador de currículo"
    )
    entrada = f"Currículo:\n{curriculo_extraido}"
    return call_agent(otimizador, entrada)

In [37]:
def agente_carta(curriculo_otimizado: str):
    redator = Agent(
        name="agente_carta",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator profissional especializado na criação de cartas de apresentação impactantes e bem estruturadas.

        Com base no currículo fornecido, redija uma carta de apresentação em português com linguagem formal e objetiva. Destaque os principais pontos fortes do candidato, como experiências relevantes, habilidades técnicas e comportamentais, conquistas e diferenciais que o tornem atrativo para o mercado de trabalho.

        Adapte o tom para ser compatível com o perfil do profissional (júnior, pleno ou sênior) e enfatize como ele pode contribuir para a empresa ou vaga desejada.

        A carta deve transmitir confiança, profissionalismo e clareza, com uma estrutura lógica e persuasiva.
        """,
        description="Agente que gera carta de apresentação"
    )
    entrada = f"Currículo:\n{curriculo_otimizado}"
    return call_agent(redator, entrada)

In [38]:
def agente_recomendador(curriculo_otimizado: str):
    recomendador = Agent(
        name="agente_recomendador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um mentor de carreira experiente, especializado em orientar profissionais em busca de crescimento e evolução.

        Com base no currículo apresentado, analise o perfil do candidato e forneça recomendações personalizadas para os próximos passos no desenvolvimento profissional, incluindo:

        - Habilidades técnicas e comportamentais que devem ser aprimoradas ou adquiridas.
        - Ferramentas, softwares ou metodologias relevantes que o candidato deve aprender para se destacar na área.
        - Tipos de projetos ou experiências práticas que podem fortalecer o currículo.
        - Recursos úteis para o aprendizado contínuo, como cursos (gratuitos ou pagos), livros, sites, comunidades ou certificações reconhecidas.

        Seja estratégico, direto e contextualize as sugestões com base no nível atual do candidato, sempre visando um avanço real e aplicável no mercado de trabalho.
        """,
        description="Agente de carreira para próximos passos"
    )
    entrada = f"Currículo:\n{curriculo_otimizado}"
    return call_agent(recomendador, entrada)

Execução principal

In [39]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

if not file_name.lower().endswith(".pdf"):
    display(Markdown("⚠️ **O arquivo enviado não é um PDF. Por favor, envie um arquivo com extensão `.pdf`.**"))
    sys.exit()

try:
    with open(file_name, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf)
        curriculo_extraido = "\n".join([page.extract_text() or "" for page in reader.pages])
except Exception as e:
    display(Markdown(f"❌ **Erro ao ler o arquivo PDF. Verifique se ele está corrompido ou é um PDF válido.**"))
    sys.exit()


print("🚀 Iniciando análise com os agentes da VerificAI...\n")

avaliacao = agente_avaliador(curriculo_extraido)
display(Markdown("## 📊 Avaliação Crítica"))
display(to_markdown(avaliacao))

otimizado = agente_otimizador(curriculo_extraido)
display(Markdown("## 📄 Currículo Otimizado"))
display(to_markdown(otimizado))

carta = agente_carta(otimizado)
display(Markdown("## ✉️ Carta de Apresentação"))
display(to_markdown(carta))

recomendacoes = agente_recomendador(otimizado)
display(Markdown("## 🧭 Próximos Passos"))
display(to_markdown(recomendacoes))

print("\n✅ Análise finalizada com sucesso!")


Saving MuriloSantos (2).pdf to MuriloSantos (2) (6).pdf
🚀 Iniciando análise com os agentes da VerificAI...



## 📊 Avaliação Crítica

> ## Análise do Currículo de Murilo Santos
> 
> **Pontos Fortes:**
> 
> *   **Objetivo Profissional:** Bem definido e alinhado com a busca por oportunidades de estágio na área de tecnologia. Demonstra proatividade e vontade de aprender.
> *   **Formação Acadêmica:**
>     *   Cursando tecnólogo em Análise e Desenvolvimento de Sistemas (FATEC Campinas) demonstra o foco na área.
>     *   O técnico em Informática para Internet (ETEC) complementa a formação, mostrando uma base sólida.
> *   **Atividades Extracurriculares:** Demonstra engajamento em atividades voluntárias, o que é um ponto positivo. As funções descritas indicam habilidades de comunicação, organização e trabalho em equipe.
> *   **Habilidades Técnicas:**
>     *   Bom conhecimento em diversas linguagens de programação (C#, Javascript, Python, Java, SQL)
>     *   Inglês intermediário é relevante para a área de tecnologia.
> *   **Certificações:**
>     *   Apresenta diversas certificações relevantes, demonstrando iniciativa em aprimorar seus conhecimentos.
> *   **Soft Skills:** Menciona soft skills importantes como resiliência, diligência, interesse e empatia.
> *   **Link para o GitHub:** Incluir o link para o GitHub é fundamental para que os recrutadores possam avaliar seus projetos e código.
> 
> **Pontos a serem aprimorados:**
> 
> *   **Formatação e Organização:**
>     *   A formatação do currículo precisa ser revisada. Há quebras de linha estranhas (ex: "Telefone:" no início) e falta de padronização na apresentação das informações.
>     *   A seção de "Atividades Extracurriculares" poderia ser renomeada para "Experiência Voluntária" para melhor compreensão.
> *   **Detalhes das Experiências:**
>     *   Na experiência como voluntário na "Nanna Salgados", seria interessante detalhar as ferramentas e tecnologias utilizadas na confecção de criativos e no gerenciamento de tráfego pago.
>     *   Na experiência na igreja, especifique quais ferramentas foram utilizadas para o gerenciamento das planilhas de dados.
> *   **Habilidades:**
>     *   Em vez de apenas listar as linguagens, tente especificar o nível de proficiência em cada uma (ex: "Javascript (React) - Nível intermediário").
> *   **Soft Skills:**
>     *   Em vez de apenas listar as soft skills, considere dar exemplos de como você as aplicou em situações reais (ex: "Resiliência: Superei desafios em projetos acadêmicos ao persistir na busca por soluções...").
> *   **Revisão Geral:**
>     *   Revise a ortografia e gramática do currículo com atenção.
> 
> **Sugestões Adicionais:**
> 
> *   **Palavras-chave:** Analise descrições de vagas de estágio que te interessam e incorpore as palavras-chave relevantes no seu currículo (ex: "Metodologias ágeis", "Scrum", "Testes unitários").
> *   **Formato do arquivo:** Salve o currículo em formato PDF para garantir que a formatação seja mantida.
> *   **Foto:** A inclusão de uma foto é opcional, mas pode ajudar a criar uma conexão visual com o recrutador. Se optar por incluir, utilize uma foto profissional.
> 
> Ao implementar essas melhorias, Murilo poderá criar um currículo mais atraente e eficaz para as oportunidades de estágio que busca.

## 📄 Currículo Otimizado

> **MURILO SANTOS**
> (19) 99900-1103 | murilosantosfvg@gmail.com | R. Carlos Roberto Ianses, 191 - Vinhedo
> 
> **OBJETIVO**
> 
> Estudante de Análise e Desenvolvimento de Sistemas em busca de oportunidade de estágio para aplicar e aprimorar habilidades técnicas em projetos desafiadores. Dedicado ao aprendizado contínuo e à contribuição proativa para o sucesso da equipe.
> 
> **FORMAÇÃO ACADÊMICA**
> 
> *   **Tecnólogo em Análise e Desenvolvimento de Sistemas** | FATEC Campinas | 2023 - Presente
> *   **Técnico em Informática para Internet** | ETEC Vasco Antônio Venchiarutti | 2020 - 2022
> 
> **EXPERIÊNCIA VOLUNTÁRIA**
> 
> *   **Nanna Salgados** | 2023 - Dez/24
> 
>     *   Criação de materiais visuais para marketing digital.
>     *   Gerenciamento de campanhas de tráfego pago.
> *   **Igreja** | 2022 - Presente
> 
>     *   Acolhimento de jovens.
>     *   Gerenciamento de planilhas de dados e suporte na organização de eventos.
>     *   Participação em programa de desenvolvimento de liderança.
> 
> **HABILIDADES TÉCNICAS**
> 
> *   **Linguagens de Programação:** C# (.NET), JavaScript (React), Python, Java, SQL
> *   **Idiomas:** Português (Fluente), Inglês (Intermediário)
> *   **Outras:** Versionamento Git
> 
> **CERTIFICAÇÕES**
> 
> *   Oracle Java Foundations | Janeiro de 2025
> *   Java e a arte da abstração | DIO | Fevereiro de 2025
> *   9° Imersão Dev | Alura | Abril de 2025
> *   Introdução ao .NET | DIO | Outubro de 2024
> *   Versionamento Git | DIO | Outubro de 2024
> *   Hardware Basics | CISCO | Maio de 2024
> 
> **SOFT SKILLS**
> 
> Resiliência | Diligência | Proatividade | Empatia
> 
> **GITHUB**
> 
> [https://github.com/MiragemS](https://github.com/MiragemS)

## ✉️ Carta de Apresentação

> [Seu Nome]
> [Seu Endereço]
> [Seu Telefone]
> [Seu Email]
> 
> [Data]
> 
> [Nome do Recrutador ou Departamento de RH]
> [Empresa]
> [Endereço da Empresa]
> 
> **Assunto: Candidatura para Estágio em Análise e Desenvolvimento de Sistemas**
> 
> Prezados(as) Senhores(as),
> 
> Escrevo para expressar meu grande interesse na oportunidade de estágio em Análise e Desenvolvimento de Sistemas em [Nome da Empresa]. Como estudante dedicado do curso de Tecnólogo em Análise e Desenvolvimento de Sistemas na FATEC Campinas, possuo uma base sólida em programação e um desejo genuíno de aplicar meus conhecimentos em projetos práticos e desafiadores.
> 
> Durante minha formação técnica em Informática para Internet na ETEC Vasco Antônio Venchiarutti, adquiri habilidades essenciais em diversas linguagens de programação, como C# (.NET), JavaScript (React), Python, Java e SQL. Além disso, possuo experiência em versionamento Git, o que me permite trabalhar de forma colaborativa e eficiente em equipe.
> 
> Minha experiência voluntária na Nanna Salgados demonstrou minha capacidade de criar materiais visuais impactantes para marketing digital e gerenciar campanhas de tráfego pago, evidenciando meu comprometimento com resultados e minha habilidade de aprendizado rápido. Adicionalmente, meu envolvimento na Igreja me proporcionou experiência em acolhimento de jovens, gerenciamento de planilhas de dados e suporte na organização de eventos, demonstrando minhas habilidades de organização, comunicação e liderança.
> 
> Sou proativo, resiliente e tenho grande facilidade em aprender novas tecnologias. Acredito que minhas habilidades técnicas, combinadas com minhas qualidades comportamentais, me tornam um candidato ideal para contribuir de forma significativa com a equipe de [Nome da Empresa].
> 
> Agradeço a atenção e coloco-me à disposição para uma entrevista, onde poderei detalhar minhas qualificações e demonstrar meu entusiasmo em fazer parte da equipe de vocês.
> 
> Atenciosamente,
> 
> Murilo Santos

## 🧭 Próximos Passos

> Murilo, seu currículo demonstra uma base sólida e proatividade, o que é ótimo para um estudante buscando estágio. Aqui estão algumas recomendações para você dar um upgrade na sua trajetória:
> 
> **Habilidades Técnicas:**
> 
> *   **Aprofunde-se em Front-end:** Você mencionou React, o que é excelente. Explore mais a fundo o ecossistema JavaScript, como Next.js ou frameworks de componentização (Styled Components, Material UI). Design responsivo (Mobile-First) também é uma habilidade valiosa.
> *   **Back-end e APIs:** Já que você tem conhecimento em C#, Python e Java, escolha uma área para se especializar. Se optar por .NET, explore o ASP.NET Core para construção de APIs RESTful. Se for Python, Django ou Flask são ótimas opções.
> *   **Banco de Dados:** SQL é fundamental, mas procure aprender sobre bancos de dados NoSQL (MongoDB, Cassandra) para ter um leque maior de opções.
> *   **Cloud:** Familiarize-se com serviços de nuvem (AWS, Azure, Google Cloud). A capacidade de implantar e gerenciar aplicações na nuvem é um grande diferencial.
> 
> **Ferramentas e Metodologias:**
> 
> *   **DevOps:** Estude sobre Integração Contínua/Entrega Contínua (CI/CD) com ferramentas como Jenkins, GitLab CI ou GitHub Actions.
> *   **Containers:** Docker e Kubernetes são cruciais para o desenvolvimento moderno. Aprenda a containerizar suas aplicações.
> *   **Metodologias Ágeis:** Aprofunde seus conhecimentos em Scrum ou Kanban.
> 
> **Projetos e Experiências Práticas:**
> 
> *   **Projetos Pessoais:** Crie projetos mais ambiciosos para o seu portfólio no GitHub. Desenvolva uma aplicação web completa (front-end, back-end e banco de dados) que resolva um problema real.
> *   **Contribuições Open Source:** Contribua para projetos open source relevantes. Isso demonstra suas habilidades e capacidade de trabalhar em equipe.
> *   **Desafios de Código:** Participe de desafios de programação em plataformas como HackerRank ou LeetCode para aprimorar suas habilidades de resolução de problemas.
> 
> **Recursos para Aprendizado Contínuo:**
> 
> *   **Cursos:**
>     *   **Udemy/Coursera:** Procure cursos específicos nas áreas que você deseja se aprofundar (ex: "React Avançado", "API REST com .NET Core").
>     *   **Alura:** Explore os cursos de desenvolvimento web e back-end.
>     *   **Rocketseat:** Oferece cursos intensivos e focados em tecnologias modernas.
> *   **Livros:**
>     *   "Clean Code" (Robert C. Martin): Melhora a qualidade do seu código.
>     *   "Design Patterns" (Erich Gamma et al.): Essencial para arquitetura de software.
> *   **Comunidades:**
>     *   Participe de fóruns e grupos de discussão online (ex: Stack Overflow, comunidades no Discord ou Telegram).
>     *   Vá a meetups e eventos de tecnologia na sua região.
> *   **Certificações:**
>     *   Além das que você já possui, considere certificações mais avançadas na área que você escolher se especializar (ex: AWS Certified Developer, Microsoft Certified Azure Developer).
> 
> **Próximos Passos Estratégicos:**
> 
> 1.  **Defina sua área de interesse:** Front-end, back-end, cloud, etc. Isso te ajudará a focar seus estudos.
> 2.  **Crie um plano de aprendizado:** Liste as habilidades que você precisa desenvolver e os recursos que vai utilizar.
> 3.  **Coloque em prática:** Desenvolva projetos pessoais para aplicar o que você está aprendendo.
> 4.  **Networking:** Conecte-se com outros profissionais da área.
> 
> Com dedicação e foco, você estará bem preparado para conseguir um estágio e construir uma carreira de sucesso na área de tecnologia.


✅ Análise finalizada com sucesso!
